In [ ]:
pip install pandas sqlalchemy pymysql


In [ ]:
pip install cryptography


In [13]:
# 1. Import libraries
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

# 2. Load environment variables
load_dotenv()
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
db = os.getenv("DB_NAME")

# 3. Create engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

# 4. rentals_month
def rentals_month(engine, month, year):
    query = f"""
        SELECT rental_id, rental_date, customer_id
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """
    return pd.read_sql(query, engine)

# 5. rental_count_month
def rental_count_month(df, month, year):
    return df.groupby('customer_id').size().reset_index(
        name=f"rentals_{month:02d}_{year}"
    )

# 6. compare_rentals
def compare_rentals(df1, df2):
    merged = pd.merge(df1, df2, on='customer_id', how='inner')
    col1 = df1.columns[1]
    col2 = df2.columns[1]
    merged['difference'] = merged[col2] - merged[col1]
    return merged

# 7. Run and compare
may_df = rentals_month(engine, 5, 2005)
june_df = rentals_month(engine, 6, 2005)

may_count = rental_count_month(may_df, 5, 2005)
june_count = rental_count_month(june_df, 6, 2005)

result = compare_rentals(may_count, june_count)
result.head()


,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
